In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [7]:

#载入数据集
mnist = input_data.read_data_sets("MNIST_data",one_hot =True)
#设置batch
batch_size = 100
#计算batch数
n_batch = mnist.train.num_examples // batch_size


#初始化权值
def w(shape):
    initial = tf.truncated_normal(shape,stddev=0.1) #生成一个截断的正太分布
    return tf.Variable(initial)

#初始化bias
def b(shape):
    initial = tf.constant(0.1, shape=shape) #生成一个截断的正太分布
    return tf.Variable(initial)


#卷积层
def conv2d(x,w):
    return tf.nn.conv2d(x,w,strides = [1,1,1,1],padding = 'SAME')

#pooling layer
def max_pool_2x2(x):
    return tf.nn.max_pool(x,ksize = [1,2,2,1],strides=[1,2,2,1],padding='SAME')

#定义两个占位符 x，y标签 输入为28x28像素的图片
x = tf.placeholder(tf.float32,[None,784])
y = tf.placeholder(tf.float32,[None,10])

#改变x的格式为4D向量 [batch,height,weigh,channel]
x_image = tf.reshape(x,[-1,28,28,1])

#初始化第一个卷积层的weights和bias
w_conv1 = w([5,5,1,32]) #5*5的32个卷积核  输入通道数是1（因为是黑白图片）
b_conv1 = b([32])

#做卷积计算和pooling计算
h_conv1 = tf.nn.relu(conv2d(x_image,w_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

#定义第二个卷积层的结构和计算
w_conv2 = w([5,5,32,64]) #输入通道32，输出64
b_conv2 = b([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1,w_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

#原图为28*28 padding为same，经过第一次卷积池化为14*14*32，经过第二次为7*7*64

#初始化第一个FC层的结构
w_fc1 = w([7*7*64,1024]) #输出设为1024
b_fc1 = b([1024])

#将第二个池化层的输出扁平为一维，再连接FC1
h_pool2_flat = tf.reshape(h_pool2,[-1,7*7*64])
#FC1的输出
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat,w_fc1) + b_fc1)

#定义dropout
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1,keep_prob)

#初始化FC2
w_fc2 = w([1024,10]) #10个输出
b_fc2 = b([10])

prediction = tf.nn.softmax(tf.matmul(h_fc1_drop,w_fc2)+b_fc2)

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels= y ,logits=prediction))

train_step = tf.train.MomentumOptimizer(learning_rate=0.001,momentum=0.1).minimize(loss)

#结果存放在一个bool型列表中 
correct_prediction = tf.equal(tf.argmax(y,1),tf.arg_max(prediction,1))#argmax返回一维向量中最大值的位置
#求准确率
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32)) #cast把第一个参数的类型转化为第二个参数的类型
#初始化
init= tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(10):
        for batch in range(n_batch):
            batch_xs,batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train_step,feed_dict = {x:batch_xs,y:batch_ys,keep_prob: 0.7})
            
        acc = sess.run(accuracy,feed_dict = {x:mnist.test.images, y:mnist.test.labels, keep_prob: 1.0})
        print("当前epoch为"+str(epoch)+", test accuracy: "+str(acc))
    

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
当前epoch为0, test accuracy: 0.3006
当前epoch为1, test accuracy: 0.493
当前epoch为2, test accuracy: 0.5831
当前epoch为3, test accuracy: 0.6144
当前epoch为4, test accuracy: 0.6332
当前epoch为5, test accuracy: 0.7079
当前epoch为6, test accuracy: 0.7276
当前epoch为7, test accuracy: 0.7369
当前epoch为8, test accuracy: 0.7414
当前epoch为9, test accuracy: 0.7468
